In [1]:
# device = 'K0303'
# active_power_variable = 'active_power'
path_base_r_string = r'D:\OneDrive - CELSIA S.A E.S.P'

%load_ext autoreload
%autoreload 2
import warnings
# warnings.filterwarnings("ignore")

In [2]:
# this cell enables project path relative imports
import sys
from pathlib import Path
path_base = Path(path_base_r_string)
project_path = path_base / 'Proyectos' / 'Eficiencia_Energetica' / 'Homecenter' / 'Informe_01'
sys.path.append(str(project_path))

In [22]:
# import all your modules here
import json
import pandas as pd
import numpy as np

from library_ubidots_v2 import Ubidots as ubi
from library_report_v2 import Processing as pro

In [4]:
# to get the missing devices we must request all the devices from the account
df_devices_acc = ubi.get_available_devices_v2(label=None, level='account', page_size=1000)
df_devices = df_devices_acc[df_devices_acc['device_name'].str.startswith('HC - ')]

In [11]:
DEVICE_IDS_TO_REQUEST = list(df_devices['device_id'])
df_vars = ubi.get_available_variables(DEVICE_IDS_TO_REQUEST)

In [14]:
whitelisted_var_labels = [
    'ea-total',
    'consumo-total-tienda',
]

In [15]:
is_whitelisted_var = df_vars['variable_label'].isin(whitelisted_var_labels)
VAR_IDS_TO_REQUEST = df_vars.loc[is_whitelisted_var, 'variable_id']
VAR_ID_TO_LABEL = dict(zip(df_vars['variable_id'], df_vars['variable_label']))

In [19]:
CHUNK_SIZE = 10
DATE_INTERVAL_REQUEST = {'start': '2022-06-01', 'end': '2022-10-31'}

df = None
lst_responses = []
for idx in range(0, ubi.ceildiv(len(VAR_IDS_TO_REQUEST), CHUNK_SIZE)):
    idx_start = idx * CHUNK_SIZE
    idx_end = (idx + 1) * CHUNK_SIZE
    id_request_subset = VAR_IDS_TO_REQUEST[idx_start:idx_end]

    response = ubi.make_request(
        id_request_subset, 
        DATE_INTERVAL_REQUEST, 
    )

    lst_responses.append(response)

df = ubi.parse_response(lst_responses, VAR_ID_TO_LABEL)
pd.to_pickle(df, project_path / "office_level_data.pkl")

In [23]:
df = pro.datetime_attributes(df)

In [24]:
df

,value,variable,device,device_name,hour,day,dow,cont_dow,week,month,year
datetime,,,,,,,,,,,
2022-10-31 00:00:00-05:00,196.700422,ea-total,hc---cali-norte,HC - Cali norte,0,31,lunes,0.000000,44,10,2022
2022-10-30 23:00:00-05:00,237.670422,ea-total,hc---cali-norte,HC - Cali norte,23,30,domingo,6.958333,43,10,2022
2022-10-30 22:00:00-05:00,213.570422,ea-total,hc---cali-norte,HC - Cali norte,22,30,domingo,6.916667,43,10,2022
2022-10-30 21:00:00-05:00,216.980422,ea-total,hc---cali-norte,HC - Cali norte,21,30,domingo,6.875000,43,10,2022
2022-10-30 20:00:00-05:00,233.000422,ea-total,hc---cali-norte,HC - Cali norte,20,30,domingo,6.833333,43,10,2022
...,...,...,...,...,...,...,...,...,...,...,...
2022-06-01 04:00:00-05:00,71.840000,ea-total,hc-san-juan,HC - San Juan,4,1,miércoles,2.166667,22,6,2022
2022-06-01 03:00:00-05:00,86.300000,ea-total,hc-san-juan,HC - San Juan,3,1,miércoles,2.125000,22,6,2022
2022-06-01 02:00:00-05:00,48.120000,ea-total,hc-san-juan,HC - San Juan,2,1,miércoles,2.083333,22,6,2022


In [25]:
df_ea = df.query("variable == 'ea-total'")

In [28]:
df_cons_monthly = (
    df_ea
    .groupby(['device_name','month'])['value']
    .sum()
    .reset_index()
)

In [29]:
df_cons_monthly

,device_name,month,value
0,HC - Barranquilla,6,229215.616260
1,HC - Barranquilla,7,227175.855247
2,HC - Barranquilla,8,236420.663175
3,HC - Barranquilla,9,226924.914946
4,HC - Barranquilla,10,157998.634683
...,...,...,...
65,HC - Tintal,6,63074.840000
66,HC - Tintal,7,59473.860000
67,HC - Tintal,8,65188.450000
68,HC - Tintal,9,62759.990000


In [30]:
df_cons_monthly_wide = df_cons_monthly.pivot(index='month', columns='device_name', values='value')

In [32]:
df_cons_monthly_wide.transpose().to_clipboard()

In [33]:
df_ea

,value,variable,device,device_name,hour,day,dow,cont_dow,week,month,year
datetime,,,,,,,,,,,
2022-10-31 00:00:00-05:00,196.700422,ea-total,hc---cali-norte,HC - Cali norte,0,31,lunes,0.000000,44,10,2022
2022-10-30 23:00:00-05:00,237.670422,ea-total,hc---cali-norte,HC - Cali norte,23,30,domingo,6.958333,43,10,2022
2022-10-30 22:00:00-05:00,213.570422,ea-total,hc---cali-norte,HC - Cali norte,22,30,domingo,6.916667,43,10,2022
2022-10-30 21:00:00-05:00,216.980422,ea-total,hc---cali-norte,HC - Cali norte,21,30,domingo,6.875000,43,10,2022
2022-10-30 20:00:00-05:00,233.000422,ea-total,hc---cali-norte,HC - Cali norte,20,30,domingo,6.833333,43,10,2022
...,...,...,...,...,...,...,...,...,...,...,...
2022-06-01 04:00:00-05:00,71.840000,ea-total,hc-san-juan,HC - San Juan,4,1,miércoles,2.166667,22,6,2022
2022-06-01 03:00:00-05:00,86.300000,ea-total,hc-san-juan,HC - San Juan,3,1,miércoles,2.125000,22,6,2022
2022-06-01 02:00:00-05:00,48.120000,ea-total,hc-san-juan,HC - San Juan,2,1,miércoles,2.083333,22,6,2022


In [36]:
(
    df_ea
    .sort_index()
    .reset_index()
    .groupby(['device_name','month'])['datetime']
    .agg(['first','last'])
    .reset_index()
).to_clipboard()